# ライブラリ読み込み

In [1]:
import pandas as pd

In [2]:
import numpy as np

# 予測結果読み込み

In [3]:
result_lstm = pd.read_csv("result_LSTM.csv")
result_lstm

,kind,date,mode_price
0,だいこん,20220502,983.71630
1,だいこん,20220506,1022.01904
2,だいこん,20220507,1016.81226
3,だいこん,20220509,1019.20280
4,だいこん,20220510,1013.12994
...,...,...,...
315,ミニトマト,20220526,136.31865
316,ミニトマト,20220527,137.25015
317,ミニトマト,20220528,140.28694
318,ミニトマト,20220530,136.14539


In [4]:
result_prophet = pd.read_csv("result_prophet.csv")
result_prophet

,kind,date,mode_price
0,だいこん,20220502,853.775550
1,だいこん,20220506,835.035522
2,だいこん,20220507,820.898149
3,だいこん,20220509,822.616543
4,だいこん,20220510,825.684007
...,...,...,...
315,ミニトマト,20220526,95.767513
316,ミニトマト,20220527,95.853024
317,ミニトマト,20220528,97.241873
318,ミニトマト,20220530,97.441669


# Ensemble

## LSTM:Prophet=5:5

In [5]:
df_result = pd.merge(result_lstm, result_prophet, how="left", on=["kind", "date"])

In [6]:
df_result["mode_price"] = (df_result["mode_price_x"] + df_result["mode_price_y"])/2
df_result

,kind,date,mode_price_x,mode_price_y,mode_price
0,だいこん,20220502,983.71630,853.775550,918.745925
1,だいこん,20220506,1022.01904,835.035522,928.527281
2,だいこん,20220507,1016.81226,820.898149,918.855204
3,だいこん,20220509,1019.20280,822.616543,920.909672
4,だいこん,20220510,1013.12994,825.684007,919.406974
...,...,...,...,...,...
315,ミニトマト,20220526,136.31865,95.767513,116.043081
316,ミニトマト,20220527,137.25015,95.853024,116.551587
317,ミニトマト,20220528,140.28694,97.241873,118.764407
318,ミニトマト,20220530,136.14539,97.441669,116.793530


In [7]:
df_result.drop(["mode_price_x", "mode_price_y"], axis=1)

,kind,date,mode_price
0,だいこん,20220502,918.745925
1,だいこん,20220506,928.527281
2,だいこん,20220507,918.855204
3,だいこん,20220509,920.909672
4,だいこん,20220510,919.406974
...,...,...,...
315,ミニトマト,20220526,116.043081
316,ミニトマト,20220527,116.551587
317,ミニトマト,20220528,118.764407
318,ミニトマト,20220530,116.793530


In [8]:
df_result.drop(["mode_price_x", "mode_price_y"], axis=1).to_csv("result_LSTM5_prophet5.csv", index=False, encoding="utf_8_sig")